In [1]:
import pandas as pd
import numpy as np
import string
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, LSTM
from sklearn import model_selection
from sklearn.metrics import accuracy_score, f1_score, roc_curve, auc

/Users/Young/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
# Date collection functions.
def convert_to_bag_of_words(text):
    list_of_letters = list(text)
    word = ''
    dictionary = dict()
    letters_list = list(string.ascii_letters)

    for i in range(len(list_of_letters)):
        if list_of_letters[i] in letters_list:
            word = word + list_of_letters[i]
        else:
            if word != 'I':
                word = word.lower()
            if word not in dictionary and word != '':
                dictionary[word] = 1
            elif word != '':
                dictionary[word] += 1
            word = ''
    return dictionary

def merge(a, b):
    for x in b.keys():
        if x in a.keys():
            a[x] = a[x] + b[x]
        else:
            a[x] = b[x]
    return a

def f(df, col_1, col_2):
    a = list(df[col_1])
    b = list(set(a))
    for x in b:
        a.remove(x)
    c = list(set(a))
    for j in c:
        a = list(df.loc[df[col_1] == j][col_2])

        for i in range(len(a)):
            if i != 0:
                a[0] = merge(a[0], a[i])

        d = list(df.loc[df[col_1] == j].index)

        for x in d:
            if d.index(x) != 0:
                df.drop(x, inplace=True)

def clean(c):
    d = list(c.values())
    limit = max(d) * 0.05

    dead_list = []

    for x in c.items():
        if x[1] <= limit:
            dead_list.append(x[0])

    for x in dead_list:
        c.pop(x)

In [3]:
# RNN type feature shaping functions.

# Convert all the text to a list of words.
def convert_to_list_of_words(text):
    list_of_letters = list(text)
    word = ''
    word_list = []
    letters_list = list(string.ascii_letters)

    for i in range(len(list_of_letters)):
        if list_of_letters[i] in letters_list:
            word = word + list_of_letters[i]
        else:
            if word != 'I':
                word = word.lower()
            if word != '':
                word_list.append(word)
            word = ''
    return word_list

# Construct the matrix for one text.
def convert_to_matrix(max_lenght, word_list, sentence):
    s = convert_to_list_of_words(sentence)
    b = word_list
    line = list(np.zeros((max_lenght+1), dtype=int))
        
    matrix = []
    for check in word_list:
        defualt_line = line[:]
        for index in range(0,len(s)):
            if check == s[index]:
                defualt_line[index] = 1
        matrix.append(defualt_line)
    return matrix

# Find the max length.
def find_max_length(text_list):
    max_lenght = 0
    for item in text_list:
        words = convert_to_list_of_words(item)
        lenght = len(words)
        if lenght > max_lenght:
            max_lenght = lenght
    return max_lenght

# Main function.
def create_RNN_feature(df, word_list):
    feature = []
    text_list = np.array(df['TEXT'])
    max_lenght = find_max_length(text_list)
    
    for item in list(df['TEXT']):
        m = convert_to_matrix(max_lenght, word_list, item)
        feature.append(m)
    return feature

In [4]:
# Load noteevents data.
df = pd.read_csv('NOTEEVENTS.csv', nrows=4000)
df.drop(columns=[
    'ROW_ID',
    'CHARTTIME',
    'CHARTDATE',
    'STORETIME',
    'CATEGORY',
    'DESCRIPTION',
    'CGID',
    'ISERROR'], inplace=True)
print('Finish loading', len(df.index), 'lines of NOTEEVENTS data.')

Finish loading 4000 lines of NOTEEVENTS data.


In [5]:
# Problem: So slow...
# Convert notes to bag of words (very slow)...
bag_of_words = []

for i in range(len(df.TEXT)):
    bag_of_words.append(convert_to_bag_of_words(df.TEXT[i]))

df['bag_of_words'] = bag_of_words
f(df, 'HADM_ID', 'bag_of_words')
print('Converting to bag of words, finished.')
print('\n')
print(df)

Converting to bag of words, finished.


      SUBJECT_ID  HADM_ID                                               TEXT  \
0          22532   167853  Admission Date:  [**2151-7-16**]       Dischar...   
1          13702   107527  Admission Date:  [**2118-6-2**]       Discharg...   
2          13702   167118  Admission Date:  [**2119-5-4**]              D...   
3          13702   196489  Admission Date:  [**2124-7-21**]              ...   
4          26880   135453  Admission Date:  [**2162-3-3**]              D...   
5          53181   170490  Admission Date:  [**2172-3-5**]              D...   
6          20646   134727  Admission Date:  [**2112-12-8**]              ...   
7          42130   114236  Admission Date:  [**2150-2-25**]              ...   
8          56174   163469  Admission Date:  [**2118-8-10**]              ...   
9          56174   189681  Admission Date:  [**2118-12-7**]              ...   
10         28063   121936  Admission Date:  [**2125-2-9**]              D...   


In [6]:
# Load admissions data.
df1 = pd.read_csv('ADMISSIONS.csv', nrows=4000)
df1.drop(columns=['ROW_ID', 'ADMITTIME', 'DISCHTIME',
       'DEATHTIME', 'ADMISSION_TYPE', 'ADMISSION_LOCATION',
       'DISCHARGE_LOCATION', 'INSURANCE', 'LANGUAGE', 'RELIGION',
       'MARITAL_STATUS', 'ETHNICITY', 'EDREGTIME', 'EDOUTTIME', 'DIAGNOSIS',
        'HAS_CHARTEVENTS_DATA'], inplace=True)
df1.dropna(inplace=True)
print('Finish loading', len(df1.index), 'lines of Admissions data.')

Finish loading 4000 lines of Admissions data.


In [7]:
# Check how many data match.
check = df1['HADM_ID'].tolist()
good_data = []

for HI in df['HADM_ID']:
    if HI in check:
        good_data.append(HI)

print('Totally', len(good_data), 'data match!')

Totally 278 data match!


In [8]:
# Filter matched data for admission (relatively slow).
for HI in df1['HADM_ID']:
    if HI not in good_data:
         df1.drop(df1.loc[df1['HADM_ID'] == HI].index, inplace=True)
df1.set_index('HADM_ID', inplace=True)
df1.sort_index(inplace=True)
df1.drop(columns=['SUBJECT_ID'], inplace=True)
df1

,HOSPITAL_EXPIRE_FLAG
HADM_ID,
100195,0
100536,0
100677,0
100680,0
100863,0
101061,0
102076,0
102248,0
102365,0


In [9]:
# Filter matched data for noteevents (relatively slow).
for HI in df['HADM_ID']:
    if HI not in good_data:
         df.drop(df.loc[df['HADM_ID'] == HI].index, inplace=True)
df.set_index('HADM_ID', inplace=True)
df.sort_index(inplace=True)
df

,SUBJECT_ID,TEXT,bag_of_words
HADM_ID,,,
100195,2489,Admission Date: [**2143-8-23**] Dischar...,"{'admission': 1, 'date': 3, 'discharge': 5, 'o..."
100536,78,Admission Date: [**2177-2-14**] Dischar...,"{'admission': 17, 'date': 5, 'discharge': 11, ..."
100677,808,Admission Date: [**2181-7-12**] ...,"{'admission': 5, 'date': 6, 'discharge': 6, 'o..."
100680,2939,Admission Date: [**2181-10-4**] Discharge...,"{'admission': 8, 'date': 4, 'discharge': 13, '..."
100863,797,Admission Date: [**2139-8-24**] ...,"{'admission': 4, 'date': 4, 'discharge': 9, 'o..."
101061,1578,Admission Date: [**2139-5-18**] ...,"{'admission': 2, 'date': 3, 'discharge': 8, 'o..."
102076,1749,Admission Date: [**2133-1-12**] Dischar...,"{'admission': 2, 'date': 3, 'discharge': 1, 'o..."
102248,3325,Admission Date: [**2159-8-14**] Discharge...,"{'admission': 4, 'date': 6, 'discharge': 9, 'o..."
102365,878,Admission Date: [**2137-10-7**] ...,"{'admission': 3, 'date': 6, 'discharge': 19, '..."


In [10]:
# Clip and find all words will be used.
total = dict()

for x in df['bag_of_words']:
    total = merge(total, x)

clean(total)
print(total)

{'admission': 1300, 'date': 1122, 'discharge': 1985, 'of': 8274, 'history': 1791, 'this': 1176, 'is': 2501, 'a': 5645, 'with': 5818, 'status': 699, 'in': 4380, 'to': 8233, 'the': 11866, 'and': 9524, 'he': 2604, 'on': 5677, 'for': 3806, 'an': 1153, 'patient': 4673, 'that': 1190, 'had': 1550, 'well': 632, 'chest': 776, 'pain': 950, 'or': 1595, 'was': 8859, 'have': 855, 'hours': 613, 'as': 2018, 'his': 2005, 'at': 2222, 'time': 796, 'after': 608, 'blood': 1885, 's': 2565, 'by': 1169, 'd': 1274, 'right': 1167, 'medications': 805, 'mg': 3820, 'day': 2110, 'no': 3066, 'first': 851, 'normal': 873, 'are': 671, 'pt': 1015, 'left': 1362, 'hospital': 1726, 'course': 658, 'stable': 698, 'which': 972, 'there': 801, 'ct': 622, 'negative': 652, 'also': 702, 'given': 729, 'care': 657, 'from': 953, 'continued': 666, 'home': 640, 'disease': 679, 'has': 925, 'up': 838, 'be': 1163, 'follow': 605, 'last': 1603, 'name': 3037, 'namepattern': 825, 'p': 1315, 'daily': 1528, 'tablet': 2538, 'o': 1220, 'c': 835,

In [11]:
# Join data.
df.drop(columns=['bag_of_words'], inplace=True)
full_data = df.join(df1)

In [12]:
full_data

,SUBJECT_ID,TEXT,HOSPITAL_EXPIRE_FLAG
HADM_ID,,,
100195,2489,Admission Date: [**2143-8-23**] Dischar...,0
100536,78,Admission Date: [**2177-2-14**] Dischar...,0
100677,808,Admission Date: [**2181-7-12**] ...,0
100680,2939,Admission Date: [**2181-10-4**] Discharge...,0
100863,797,Admission Date: [**2139-8-24**] ...,0
101061,1578,Admission Date: [**2139-5-18**] ...,0
102076,1749,Admission Date: [**2133-1-12**] Dischar...,0
102248,3325,Admission Date: [**2159-8-14**] Discharge...,0
102365,878,Admission Date: [**2137-10-7**] ...,0


In [13]:
############################################# Data dispose over. #####################################################
######################################################################################################################

In [14]:
# Create word list.
word_list = list(total.keys())
word_list

['admission',
 'date',
 'discharge',
 'of',
 'history',
 'this',
 'is',
 'a',
 'with',
 'status',
 'in',
 'to',
 'the',
 'and',
 'he',
 'on',
 'for',
 'an',
 'patient',
 'that',
 'had',
 'well',
 'chest',
 'pain',
 'or',
 'was',
 'have',
 'hours',
 'as',
 'his',
 'at',
 'time',
 'after',
 'blood',
 's',
 'by',
 'd',
 'right',
 'medications',
 'mg',
 'day',
 'no',
 'first',
 'normal',
 'are',
 'pt',
 'left',
 'hospital',
 'course',
 'stable',
 'which',
 'there',
 'ct',
 'negative',
 'also',
 'given',
 'care',
 'from',
 'continued',
 'home',
 'disease',
 'has',
 'up',
 'be',
 'follow',
 'last',
 'name',
 'namepattern',
 'p',
 'daily',
 'tablet',
 'o',
 'c',
 'one',
 'but',
 'not',
 'q',
 'h',
 'two',
 'her',
 'were',
 'times',
 'showed',
 'x',
 'days',
 'she',
 'am',
 'pm',
 'sig',
 'po',
 'you',
 'your']

In [15]:
# Create feature and label.
Y = list(full_data['HOSPITAL_EXPIRE_FLAG'])
feature_data = full_data.drop(columns=['HOSPITAL_EXPIRE_FLAG'])
X = np.array(create_RNN_feature(feature_data, word_list))

In [16]:
X.shape

(278, 92, 4119)

In [17]:
X_train = X
y_train = Y

In [18]:
# input shape: (position_len, volc_len)

In [19]:
model = Sequential()

model.add(LSTM(1, input_shape=(X_train.shape[1:]), activation='relu', return_sequences=False))

model.add(Dense(1, input_shape = X.shape[1:]))
model.add(Activation('sigmoid'))

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.fit(X_train, y_train, epochs=3)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Epoch 1/3
278/278 [==============================] - 9s 31ms/sample - loss: 0.6847 - acc: 0.8957
Epoch 2/3
278/278 [==============================] - 4s 16ms/sample - loss: 0.6746 - acc: 0.8957
Epoch 3/3
278/278 [==============================] - 4s 15ms/sample - loss: 0.6629 - acc: 0.8957
